# Setup

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from transformers import pipeline

In [2]:
books = pd.read_csv('data/books_with_categories.csv')

# Sentiment Analysis

In [3]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", device='mps', top_k=None)

Device set to use mps


In [20]:
classifier(books['description'][0])[0][0]

{'label': 'fear', 'score': 0.6548408269882202}

In [14]:
def split_text(text: str) -> list[str]:
    return text.split('. ')

In [15]:
s = split_text(books['description'][0])
s

['A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives',
 'John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers',
 'It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up',
 'Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist',
 'He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption',
 'Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the wo

In [16]:
predictions = classifier(s)

In [5]:
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
isbn = []
emotion_score = {label: [] for label in emotion_labels}

In [19]:
def calculate_max_emotion_scores(predictions_list: list[dict]) -> dict:
    max_scores = {label: [] for label in emotion_labels}
    for prediction in predictions_list:
        sorted_prediction = sorted(prediction, key=lambda x: x["label"])
        for idx, label in enumerate(emotion_labels):
            max_scores[label].append(sorted_prediction[idx]["score"])
    return {label: np.max(scores) for label, scores in max_scores.items()}


In [54]:
def calculate_max_score(predictions_list: list[dict]) -> dict:
    max_scores = {label: -np.inf for label in emotion_labels}
    for prediction in predictions_list:
        for label_score in prediction:
            label, score = label_score['label'], label_score['score']
            if score > max_scores[label]:
                max_scores[label] = score
    return max_scores

In [7]:
for i in tqdm(range(len(books))):
    isbn.append(books['isbn13'][i])
    sentences = split_text(books['description'][i])
    predictions = classifier(sentences)
    ms = calculate_max_score(predictions)
    for l in emotion_labels:
        emotion_score[l].append(ms[l])

100%|██████████| 5197/5197 [05:46<00:00, 14.99it/s]


In [8]:
emotions_df = pd.DataFrame(emotion_score)
emotions_df['isbn13'] = isbn
books = pd.merge(books, emotions_df, on='isbn13')

In [59]:
books.to_csv('data/books_with_emotions.csv', index=False)